#### verify_masks
This script verifies that all images in a dataset directory have a corresponding mask.  Images that do not have a mask are uploaded to LabelBox.

In [1]:
import os
from labelbox_class import *

In [ ]:
## LabelBox Params
api_key = ''
proj_name = 'Birds_Retry'
ontology_name = 'Birds_Retry'

# TXT file to be made with project and ontology IDs within it
proj_id_txt = ".\\..\\data\\projects\\"+project_name+".txt"

## Dataset Directory
dataset_dir = '.\\..\\data\\datasets\\birds_dataset\\'
masks_dir = os.path.join(dataset_dir, 'masks')
images_dir = os.path.join(dataset_dir, 'raw')

In [ ]:
no_mask_ims = []
for classes in os.listdir(images_dir):
    if classes != '.gitkeep':
        masks = [x.replace('.png', '.jpg') for x in os.listdir(os.path.join(masks_dir, classes))]
        for im in os.listdir(os.path.join(images_dir, classes)):
            if im not in masks:
                no_mask_ims.append(os.path.join(images_dir, classes, im))

In [ ]:
labels = LabelBox(api_key=api_key, img_lst=no_mask_ims, project_name=proj_name, ontology_name=ontology_name, proj_id=None, directory_path=None):

In [ ]:
# Finds all images that could not be uploaded to LabelBox
rm_lst = []
for error in tqdm(labels.find_error_sets()):
    data_lst = [ os.path.join(raw_dir, error.replace("-s ", "'s "),x) for x in os.listdir(os.path.join(raw_dir, error.replace("-s ", "'s ")))]
    rm_lst.append(labels.labelbox_dataset_lst(error.replace("-s ", "'s "), data_lst))

In [ ]:
# Removes images that could not be uploaded from the dataset 
for im_lst in rm_lst:
    for im in im_lst:
        print(im)
        os.remove(im)

In [ ]:
# Creates TXT file with project and ontolgy IDs
if os.path.exists(proj_id_txt):
    valid_path = False
    count = 0
    while not valid_path:
        new_name =  proj_id_txt.split(".txt")[0] + str(count) + ".txt"
        if not os.path.exists(new_name):
            valid_path = True
    proj_id_txt = new_name
    
with open(proj_id_txt, 'w') as f:
    f.write("Project_name:"+project_name+", Project_id:"+labels.project.uid+"\n"+
            "Ontology_name:"+ontology_name+", Ontology_id:"+labels.ontology)
print("Project ID: ",labels.project.uid)
print("Ontology ID: ", labels.ontology)
print("File made at: ", proj_id_txt)